In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
train = pd.read_csv("../input/osic-pulmonary-fibrosis-progression/train.csv")
test = pd.read_csv("../input/osic-pulmonary-fibrosis-progression/test.csv")
submission = pd.read_csv("../input/osic-pulmonary-fibrosis-progression/sample_submission.csv")

In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
test.head()

In [ ]:
train.drop_duplicates(keep=False, inplace=True, subset=['Patient', 'Weeks'])

In [ ]:
submission.head(10)

In [ ]:
submission['Patient'] = submission['Patient_Week'].apply(lambda x: x.split("_")[0])

In [ ]:
submission['Weeks'] = submission['Patient_Week'].apply(lambda x: x.split("_")[-1])

In [ ]:
submission

In [ ]:
submission = submission[['Patient', 'Weeks', 'Confidence', 'Patient_Week']]

In [ ]:
submission = submission.merge(test.drop('Weeks', axis=1), on='Patient')

In [ ]:
submission

In [ ]:
train['Dataset'] = 'train'
test['Dataset'] = 'test'
submission['Dataset'] = 'submission'

In [ ]:
all_data = train.append([test, submission])

In [ ]:
all_data = all_data.reset_index()

In [ ]:
all_data = all_data.drop(columns=['index'])

In [ ]:
all_data

In [ ]:
all_data = pd.concat([
    all_data,
    pd.get_dummies(all_data.Sex),
    pd.get_dummies(all_data.SmokingStatus)
], axis=1)

In [ ]:
all_data = all_data.drop(columns=['Sex', 'SmokingStatus'])

In [ ]:
all_data

In [ ]:
def scale_feature(series):
    return (series - series.min()) / (series.max() - series.min())

all_data['Percent'] = scale_feature(all_data['Percent'])
all_data['Age'] = scale_feature(all_data['Age'])

In [ ]:
feature_columns = [
    'Percent',
    'Age',
    'Female',
    'Male', 
    'Currently smokes',
    'Ex-smoker',
    'Never smoked',
]

In [ ]:
train = all_data.loc[all_data.Dataset == 'train']
test = all_data.loc[all_data.Dataset == 'test']
submission = all_data.loc[all_data.Dataset == 'submission']

In [ ]:
feature_columns

In [ ]:
train[feature_columns].head()

In [ ]:
train['FVC']

In [ ]:
import sklearn
from sklearn import linear_model

In [ ]:
model = linear_model.ARDRegression()

In [ ]:
model.fit(train[feature_columns],train['FVC'])

In [ ]:
plt.bar(train[feature_columns].columns.values, model.coef_) 
plt.xticks(rotation=90)

In [ ]:
predictions = model.predict(train[feature_columns])
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(train['FVC'], predictions) 
print(mae)

In [ ]:
train['prediction'] = predictions

In [ ]:
plt.scatter(predictions, train['FVC'])

In [ ]:
submission[feature_columns].head()

In [ ]:
sub_predictions = model.predict(submission[feature_columns])
submission['FVC'] = sub_predictions


In [ ]:
submission = submission[['Patient_Week', 'FVC']]

submission['Confidence'] = 285

In [ ]:
submission.to_csv('submission11.csv', index=False)